In [55]:
import re
import csv
from math import log, exp
from collections import Counter


def start():
    train_datas = open_csv()
    test_data = input()
    prob = naive_bayes(train_datas, test_data, 0.5, 0.5)

    print(f'{test_data}가 부정적일 확률 : {prob[0]}, 긍정적일 확률 : {prob[1]}')
    
def open_csv():
    f = open('./data/ratings.txt', 'r', encoding='utf-8')
    csvreader = csv.reader(f)

    pos_doc = []
    neg_doc = []

    # 첫줄은 헤더라서 스킵
    next(csvreader)
    for line in csvreader:
        if line[-1][-1] == '1':
            pos_doc.append(line[0][:-2])
        else:
            neg_doc.append(line[0][:-2])

    train_datas = [[], []]
    train_datas[0] = neg_doc
    train_datas[1] = pos_doc

    # 리스트 형태는 토큰화하기가 어렵기 때문에, 전부 조인을 통해서 하나의 문자열로 만들어준다
    return [' '.join(train_datas[0]), ' '.join(train_datas[1])]

def calculate_doc_prob(train_data, test_data, nowords_weight):
    sw_train_data = re.compile('[^\w]').sub(' ', train_data.lower())
    sw_train_token = sw_train_data.split()
    train_vector = dict(Counter(sw_train_token))

    sw_test_data = re.compile('[^\w]').sub(' ', test_data.lower())
    sw_test_token = sw_test_data.split()
    test_vector = dict(Counter(sw_test_token))

    log_prob = 0

    total_wc = len(sw_train_token)

    for word in test_vector:
        if word in train_vector:
            log_prob += log(train_vector[word]/total_wc)
        else:
            log_prob += log(nowords_weight/total_wc)

    return log_prob

def naive_bayes(train_datas, test_data, pos_prob, neg_prob):
    test_pos_prob = calculate_doc_prob(train_datas[1], test_data, 0.1) + log(pos_prob) 
    test_neg_prob = calculate_doc_prob(train_datas[0], test_data, 0.1) + log(neg_prob) 

    maxprob = max(test_neg_prob, test_pos_prob) 
    test_pos_prob -= maxprob 
    test_neg_prob -= maxprob
    test_pos_prob = exp(test_pos_prob)
    test_neg_prob = exp(test_neg_prob) 
    normalized_prob = [test_neg_prob/(test_pos_prob+test_neg_prob), test_pos_prob/(test_pos_prob+test_neg_prob)]

    return normalized_prob

start()

스토리가 점점 산으로가네..
스토리가 점점 산으로가네..가 부정적일 확률 : 0.981757353504198, 긍정적일 확률 : 0.018242646495802057
